In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df_meta = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_meta_cleaned_ver3.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_cluster = pd.read_csv('C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/code/analysis/inventory_cluster_classification/movie_meta_with_cluster.csv',engine='python',encoding='utf-8')

In [3]:
df_download.columns

Index(['customer_id', 'item_id', 'down_year', 'down_date', 'weekday',
       'inf_year', 'down_price'],
      dtype='object')

In [4]:
df_meta.columns

Index(['movie_id', 'title', 'release_year', 'release_date', 'runtime',
       'mpa_rating', 'imdb_score', 'dvd_sales', 'blu_sales', 'total_sales',
       'legs', 'share', 'inf_income_usa', 'theater_opening', 'theater_total',
       'series', 'genre', 'src', 'creative_type', 'country', 'director',
       'actor', 'writer', 'description', 'synop', 'contract_price',
       'studio_score', 'price_class', 'language', 'awards', 'poster',
       'metascore', 'release_dvd', 'big_awards_num', 'awards_win_num',
       'awards_nomin_num', 'reviews_users', 'reviews_critics', 'prd_mthd',
       'prd_company', 'budget', 'series_new', 'kwrds', 'actor_1', 'actor_2',
       'actor_3', 'actor_4', 'income_opening', 'votes', 'income_usa',
       'income_int', 'income_ww', 'item_id', 'inv_exist', 'contract_year',
       'inf', 'genre_action', 'genre_adventure', 'genre_drama', 'genre_sci-fi',
       'genre_fantasy', 'genre_thriller', 'genre_animation', 'genre_family',
       'genre_musical', 'genre_comedy',

In [5]:
df_inventory = df_meta[df_meta['inv_exist']==1]
df_inventory = df_inventory.reset_index().drop('index', axis=1)
df_inventory

,movie_id,title,release_year,release_date,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,...,genre_reality-tv,genre_film-noir,genre_talk-show,country_1,country_2,country_3,dvd_over_income,movie_down_sales,contract_price_inf,net_profit
0,tt2395427,Avengers: Age of Ultron,2015,01 May 2015,141.0,PG-13,7.3,33569045.0,55802159.0,89371204.0,...,0.0,0.0,0.0,USA,.,.,0.07313423932087944,1042.2553871899931,755.7743092899998,286.48107789999335
1,tt4154756,Avengers: Infinity War,2018,27 Apr 2018,156.0,PG-13,8.5,23700665.0,77736363.0,101437028.0,...,0.0,0.0,0.0,USA,.,.,0.034914738435503155,161.0,1000.0,-839.0
2,tt0449088,Pirates of the Caribbean: At World's End,2007,25 May 2007,167.0,PG-13,7.1,314040375.0,7640482.0,321680857.0,...,0.0,0.0,0.0,USA,.,.,1.0149309794271015,1019.2433991229697,767.8666982386397,251.37670088432992
3,tt0974015,Justice League,2017,17 Nov 2017,121.0,PG-13,6.4,19449630.0,40475432.0,59925062.0,...,0.0,0.0,0.0,USA,Canada,UK,0.08492387237781913,375.42999999999705,745.3299999999999,-369.9000000000029
4,tt2379713,Spectre,2015,06 Nov 2015,148.0,PG-13,6.8,12917812.0,24092452.0,37010264.0,...,0.0,0.0,0.0,UK,USA,Austria,0.064565114413192,472.1095652749993,755.7743092899998,-283.6647440150005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,tt4846340,Hidden Figures,2016,06 Jan 2017,126.0,PG,7.8,18636040.0,11544851.0,30180891.0,...,0.0,0.0,0.0,USA,.,.,0.10987759034197629,590.6478119999944,1034.2729999999997,-443.6251880000053
102,tt6644200,A Quiet Place,2018,06 Apr 2018,95.0,PG-13,7.5,6628735.0,10373895.0,17002630.0,...,0.0,0.0,0.0,USA,.,.,0.035254660432006465,159.0,920.0,-761.0
103,tt5052448,Get Out,2017,24 Feb 2017,104.0,R,7.7,7363633.0,6449455.0,13813088.0,...,0.0,0.0,0.0,USA,Japan,.,0.04182915918887264,579.141599999995,939.3199999999999,-360.1784000000049
104,tt0121766,Star Wars: Episode III - Revenge of the Sith,2005,19 May 2005,139.0,PG-13,7.5,.,.,.,...,0.0,0.0,0.0,USA,.,.,.,868.3820114523724,967.7224142185597,-99.34040276618737


In [6]:
df_inventory = df_inventory.merge(df_cluster.loc[:,['movie_id','cluster_kmeans']],on='movie_id', how='left')

In [7]:
df_inventory['cluster_kmeans'].unique()

array([22,  5,  2, 18,  9,  8, 19, 21], dtype=int64)

In [8]:
df_download_cluster = df_download.merge(df_cluster.loc[:,['item_id','cluster_kmeans']],on='item_id', how='left')

In [11]:
grouped_df = df_download.groupby(['customer_id'])['item_id']

df_item = []
df_key = []
for key, item in grouped_df:
    df_item.append(list(grouped_df.get_group(key)))
    df_key.append(key)
    
# Items 중 index 0번이 갖고 있으면 True, 아니면 False
enc = TransactionEncoder()
df_asso_enc = enc.fit_transform(X = df_item)
# DataFrame으로 변환
df_asso = pd.DataFrame(df_asso_enc, columns = enc.columns_, index=df_key)
df_asso.head(10)

,I-1001,I-1002,I-1003,I-1004,I-1005,I-1006,I-1007,I-1008,I-1009,I-1010,...,I-1097,I-1098,I-1099,I-1100,I-1101,I-1102,I-1103,I-1104,I-1105,I-1106
C5001,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
C5002,False,False,True,False,True,True,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
C5003,True,True,True,True,True,True,True,False,True,False,...,False,False,True,False,False,True,False,False,False,False
C5004,True,True,True,True,True,True,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
C5005,True,True,True,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C5006,False,True,False,True,True,True,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
C5007,True,True,True,False,True,True,False,True,True,True,...,False,False,False,True,False,False,False,False,False,False
C5008,True,True,True,False,False,False,True,True,True,True,...,False,False,False,False,False,True,False,False,False,False
C5009,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C5010,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
# grouped_df = df_download_cluster.groupby(['customer_id'])['cluster_kmeans']

# df_item = []
# df_key = []
# for key, item in grouped_df:
#     df_item.append(list(grouped_df.get_group(key)))
#     df_key.append(key)
    
# # Items 중 index 0번이 갖고 있으면 True, 아니면 False
# enc = TransactionEncoder()
# df_asso_enc = enc.fit_transform(X = df_item)
# # DataFrame으로 변환
# df_asso = pd.DataFrame(df_asso_enc, columns = enc.columns_, index=df_key)
# df_asso.head(10)

,2,5,8,9,18,19,21,22
C5001,True,True,True,True,False,False,False,True
C5002,True,True,False,True,True,True,False,True
C5003,True,True,False,True,True,True,True,True
C5004,True,True,True,True,False,True,True,True
C5005,True,True,True,True,False,True,True,True
C5006,True,True,True,True,False,True,True,True
C5007,True,True,True,True,True,True,True,True
C5008,True,True,True,True,True,True,True,True
C5009,False,True,False,True,False,False,False,False
C5010,True,True,True,True,False,True,True,True


In [12]:
# 지지도 계산
df_freq = apriori(df_asso, min_support=0.1, use_colnames=True)
df_freq.round(3).sort_values(by='support',ascending=False)

,support,itemsets
41,0.448,(I-1042)
44,0.445,(I-1045)
43,0.444,(I-1044)
42,0.444,(I-1043)
56,0.439,(I-1057)
...,...,...
2616,0.100,"(I-1008, I-1015, I-1057)"
2027,0.100,"(I-1046, I-1006, I-1041)"
3745,0.100,"(I-1018, I-1015, I-1016)"
1746,0.100,"(I-1011, I-1012, I-1003)"


In [64]:
df_freq.round(3).head(10).sort_values(by='support',ascending=False)

,support,itemsets
1,0.991,(5)
0,0.985,(2)
8,0.976,"(2, 5)"
3,0.944,(9)
7,0.930,(22)
5,0.852,(19)
4,0.722,(18)
2,0.720,(8)
9,0.715,"(8, 2)"
6,0.713,(21)


In [39]:
# 기준 = 신뢰도, 최소신뢰도 = 0.5
df_asso_rule = association_rules(df_freq, metric='confidence', min_threshold=0.5)
df_asso_rule.round(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(I-1001),(I-1002),0.345,0.358,0.173,0.502,1.403,0.050,1.289
1,(I-1001),(I-1008),0.345,0.422,0.182,0.528,1.252,0.037,1.225
2,(I-1001),(I-1009),0.345,0.397,0.176,0.511,1.289,0.040,1.234
3,(I-1001),(I-1010),0.345,0.408,0.177,0.512,1.254,0.036,1.213
4,(I-1001),(I-1012),0.345,0.421,0.183,0.529,1.256,0.037,1.229
...,...,...,...,...,...,...,...,...,...
13979,"(I-1044, I-1045, I-1041)","(I-1042, I-1043)",0.188,0.253,0.111,0.589,2.331,0.063,1.819
13980,"(I-1042, I-1043, I-1045)","(I-1044, I-1041)",0.182,0.250,0.111,0.612,2.448,0.066,1.932
13981,"(I-1042, I-1043, I-1041)","(I-1044, I-1045)",0.164,0.276,0.111,0.679,2.461,0.066,2.253
13982,"(I-1042, I-1045, I-1041)","(I-1044, I-1043)",0.170,0.264,0.111,0.652,2.467,0.066,2.116


In [40]:
df_asso_rule = df_asso_rule.sort_values(by='confidence', ascending=False)

In [41]:
df_asso_rule.index

Int64Index([13898, 13103, 13043, 13028, 13057, 13730, 13853, 13895, 13776,
            13823,
            ...
            11964,  8212,  8500, 10089,  6310,  8540,  5011,    50,  6235,
             8630],
           dtype='int64', length=13984)

In [52]:
for i in df_asso_rule.index:
    df_asso_rule['antecedents'].loc[i] = list(df_asso_rule['antecedents'].loc[i])
    df_asso_rule['consequents'].loc[i] = list(df_asso_rule['consequents'].loc[i])

C:\Users\cresc\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
df_asso

In [50]:
for i in df_asso_rule.index:
    if 'I-1013' in df_asso_rule['antecedents'].loc[i]:
        print(list(df_asso_rule['consequents'].loc[i]))

['I-1044']
['I-1044']
['I-1044']
['I-1045']
['I-1044']
['I-1045']
['I-1044']
['I-1045']
['I-1044']
['I-1016']
['I-1044']
['I-1045']
['I-1012']
['I-1044']
['I-1045']
['I-1045']
['I-1016']
['I-1016']
['I-1045']
['I-1016']
['I-1045']
['I-1044']
['I-1045']
['I-1045']
['I-1044']
['I-1044']
['I-1012']
['I-1016']
['I-1044']
['I-1044']
['I-1045']
['I-1044']
['I-1016']
['I-1044']
['I-1016']
['I-1016']
['I-1044']
['I-1016']
['I-1045']
['I-1012']
['I-1045']
['I-1045']
['I-1016']
['I-1044']
['I-1016']
['I-1044']
['I-1044']
['I-1010']
['I-1016']
['I-1016']
['I-1045']
['I-1012']
['I-1045']
['I-1045']
['I-1016']
['I-1044']
['I-1012']
['I-1045']
['I-1016']
['I-1044']
['I-1045']
['I-1044']
['I-1012']
['I-1044']
['I-1010']
['I-1045']
['I-1044']
['I-1010']
['I-1010']
['I-1012']
['I-1044']
['I-1016']
['I-1007']
['I-1044']
['I-1012']
['I-1016']
['I-1012']
['I-1045']
['I-1016']
['I-1044']
['I-1045']
['I-1016']
['I-1012']
['I-1012']
['I-1016']
['I-1012']
['I-1045']
['I-1016']
['I-1016']
['I-1012']
['I-1012']

['I-1044', 'I-1045']
['I-1008']
['I-1008']
['I-1010', 'I-1045']
['I-1006', 'I-1016']
['I-1044', 'I-1010']
['I-1012', 'I-1016']
['I-1016']
['I-1045']
['I-1045', 'I-1007']
['I-1016', 'I-1045']
['I-1008']
['I-1007']
['I-1006']
['I-1010', 'I-1016']
['I-1010']
['I-1009']
['I-1006']
['I-1006', 'I-1045']
['I-1044', 'I-1006']
['I-1006', 'I-1010']
['I-1007']
['I-1044', 'I-1007']
['I-1010']
['I-1044', 'I-1043']
['I-1044', 'I-1045']
['I-1016']
['I-1041']
['I-1010']
['I-1016']
['I-1009']
['I-1044']
['I-1007']
['I-1044']
['I-1016', 'I-1007']
['I-1010']
['I-1006']
['I-1016']
['I-1043']
['I-1045']
['I-1006']
['I-1010', 'I-1045']
['I-1008']
['I-1044']
['I-1010']
['I-1043', 'I-1045']
['I-1044', 'I-1016']
['I-1016', 'I-1045']
['I-1043', 'I-1016']
['I-1012', 'I-1007']
['I-1012', 'I-1045']
['I-1044']
['I-1010', 'I-1007']
['I-1044', 'I-1007']
['I-1006', 'I-1016']
['I-1043']
['I-1045']
['I-1010']
['I-1042']
['I-1044']
['I-1010', 'I-1016']
['I-1010']
['I-1009']
['I-1045']
['I-1016']
['I-1009']
['I-1008']
['I

['I-1002']
['I-1066']
['I-1042', 'I-1045']
['I-1052']
['I-1044', 'I-1010']
['I-1052']
['I-1048']
['I-1051']
['I-1066']
['I-1010', 'I-1007']
['I-1045', 'I-1016']
['I-1043', 'I-1045']
['I-1010', 'I-1045']
['I-1002']
['I-1008', 'I-1012']
['I-1052']
['I-1002']
['I-1044', 'I-1016']
['I-1047']
['I-1047']
['I-1052']
['I-1031']
['I-1066']
['I-1066']
['I-1057']
['I-1040']
['I-1044', 'I-1043']
['I-1044', 'I-1006']
['I-1047']
['I-1048']
['I-1057']
['I-1047']
['I-1018']
['I-1044', 'I-1010']
['I-1044', 'I-1006']
['I-1050']
['I-1042', 'I-1045']
['I-1052']
['I-1045', 'I-1016']
['I-1010', 'I-1045']
['I-1043', 'I-1045']
['I-1066']
['I-1049']
['I-1010', 'I-1012']
['I-1002']
['I-1008', 'I-1016']
['I-1044', 'I-1016']
['I-1052']
['I-1045', 'I-1016']
['I-1006', 'I-1045']
['I-1052']
['I-1066']
['I-1047']
['I-1043', 'I-1012']
['I-1012', 'I-1045']
['I-1051']
['I-1066']
['I-1050']
['I-1044', 'I-1006']
['I-1044', 'I-1042']
['I-1044', 'I-1007']
['I-1057']
['I-1018']
['I-1010', 'I-1012']
['I-1006', 'I-1010']
['I-1

In [53]:
df_asso_rule.to_csv('../../data/cleaned/asso_rule.csv', header=True, index=False)

In [75]:
len(df_asso_rule['antecedents'].unique())

28

In [76]:
len(df_asso_rule['consequents'].unique())

18

In [11]:
df_asso_rule['consequents']

0                (I-1002)
1                (I-1008)
2                (I-1009)
3                (I-1010)
4                (I-1012)
               ...       
13979    (I-1044, I-1041)
13980    (I-1045, I-1042)
13981    (I-1045, I-1043)
13982    (I-1045, I-1041)
13983    (I-1045, I-1044)
Name: consequents, Length: 13984, dtype: object

In [12]:
df_meta[df_meta['item_id']=='I-1044']

,movie_id,title,release_year,release_date,runtime,mpa_rating,imdb_score,dvd_sales,blu_sales,total_sales,...,genre_reality-tv,genre_film-noir,genre_talk-show,country_1,country_2,country_3,dvd_over_income,movie_down_sales,contract_price_inf,net_profit
16,tt2310332,The Hobbit: The Battle of the Five Armies,2014,17 Dec 2014,144.0,PG-13,7.4,29485612.0,48209740.0,77695352.0,...,0.0,0.0,0.0,New Zealand,USA,.,0.11557555856858896,1172.6060738468027,967.7224142185597,204.88365962824298


In [111]:
inv_same_clus = ['I-1048', 'I-1097', 'I-1007', 'I-1092', 'I-1062']
watched = ['I-1005','I-1010','I-1012']


def asso_rule_filter(inv_same_clus, watched):
    import pandas as pd
    from itertools import combinations

    asso_df = pd.read_csv("C:/Users/cresc/PycharmProjects/POSCO-Academy-A3_/proj_bigdata/code/analysis/inventory_cluster_classification/asso_rule.csv", engine="python")
    
    antecedents = []    
    for i in range(len(watched)+1):
        antecedents.append(list(combinations(watched,i)))
    print(antecedents)
    
    consequents = []
#     print(str(list(antecedents[1][0])) == str(asso_df.loc[11086,'antecedents']))
    for c in antecedents:
        for ids in list(c):
            for i in asso_df.index:
                if str(list(ids)) == str(asso_df.loc[i,'antecedents']):
                    consequents.append(asso_df.loc[i,'consequents'])
    print(consequents)
    
    inv_same_clus_0 = []
    inv_same_clus_1 = []
    for ids in consequents:
        for inv_ids in inv_same_clus:
            if ids == str(list(inv_ids)):
                if inv_ids in inv_same_clus_0 + inv_same_clus_1:
                    pass
                else:
                    inv_same_clus_0.append(inv_ids)
            if ids != str(list(inv_ids)):
                if inv_ids in inv_same_clus_0 + inv_same_clus_1 :
                    pass
                else:
                    inv_same_clus_1.append(inv_ids)
                
    if inv_same_clus_0 == []:
        return inv_same_clus

    return inv_same_clus_0 + inv_same_clus_1

asso_rule_filter(inv_same_clus, watched)

[[()], [('I-1005',), ('I-1010',), ('I-1012',)], [('I-1005', 'I-1010'), ('I-1005', 'I-1012'), ('I-1010', 'I-1012')], [('I-1005', 'I-1010', 'I-1012')]]
["['I-1002']", "['I-1012']", "['I-1010']", "['I-1008']", "['I-1011']", "['I-1015']", "['I-1016']", "['I-1036']", "['I-1009']", "['I-1007']", "['I-1013']", "['I-1012']", "['I-1013']", "['I-1044']", "['I-1007']", "['I-1016']", "['I-1045']", "['I-1043']", "['I-1042']", "['I-1008']", "['I-1006']", "['I-1009']", "['I-1041']", "['I-1057']", "['I-1008']", "['I-1010']", "['I-1013']", "['I-1045']", "['I-1043']", "['I-1016']", "['I-1044']", "['I-1007']", "['I-1006']", "['I-1042']", "['I-1041']", "['I-1009']", "['I-1066']", "['I-1057']", "['I-1012']", "['I-1010']", "['I-1016']", "['I-1013']", "['I-1044']", "['I-1008']", "['I-1007']", "['I-1043']", "['I-1045']", "['I-1006']", "['I-1009']", "['I-1042']", "['I-1041']", "['I-1047']", "['I-1050']", "['I-1031']", "['I-1066']", "['I-1057']", "['I-1018']", "['I-1013', 'I-1016']", "['I-1067']", "['I-1002']",

['I-1048', 'I-1097', 'I-1007', 'I-1092', 'I-1062']

In [109]:
a = ['1','2']
b = [2,3]
a+b

['1', '2', 2, 3]

In [60]:
from itertools import combinations

a = [1,2,3,4]
list(combinations(a,len(a)))

[(1, 2, 3, 4)]